In [34]:
import warnings
warnings.filterwarnings('ignore')


import sys
import random
import numpy as np

import pyodbc 
from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#Conectando com o Impala
pyodbc.autocommit = True
conn_impala = pyodbc.connect('DSN=Sample Cloudera Impala DSN 64', autocommit=True)
cursor_impala = conn_impala.cursor()

In [5]:
cursor_impala.execute('show databases; ')

In [3]:
def as_pandas(cursor_impala):
    names = [metadata[0] for metadata in cursor_impala.description]
    return pd.DataFrame([dict(zip(names, row)) for row in cursor_impala], columns=names)

In [6]:
as_pandas(cursor_impala)

,name,comment
0,_impala_builtins,System database for Impala builtin functions
1,default,Default Hive database
2,teste,
3,voos_brasil_labdata,


In [7]:
cursor_impala.execute('use voos_brasil_labdata; ')

In [ ]:
cursor_impala.execute('show tables; ')
as_pandas(cursor_impala)

In [ ]:
cursor_impala.execute('select cod_aut_di, cod_tipo_linha, cod_aero_org, cod_aero_dest, flg_atraso_partida, flg_atraso_chegada, feriado_federal_partida, feriado_federal_chegada, feriado_estadual_partida, feriado_estadual_chegada, feriado_municipal_partida, feriado_municipal_chegada, feriado_facultativo_partida, feriado_facultativo_chegada, carnaval_partida, carnaval_chegada, dia_da_semana_partida, dia_da_semana_chegada, final_de_semana_partida, final_de_semana_chegada, altitudeicao_origem_moda, direcaovento_origem_moda, pressaoatmestacao_origem_moda, tempbulboseco_origem_moda, tempbulboumido_origem_moda, umidaderelativa_origem_moda, velocidadeventonebulosidade_origem_moda, altitudeicao_dest_moda, direcaovento_dest_moda, pressaoatmestacao_dest_moda, tempbulboseco_dest_moda, tempbulboumido_dest_moda, umidaderelativa_dest_moda, velocidadeventonebulosidade_dest_moda, altitudeicao_origem_media, direcaovento_origem_media, pressaoatmestacao_origem_media, tempbulboseco_origem_media, tempbulboumido_origem_media, umidaderelativa_origem_media, velocidadeventonebulosidade_origem_media, altitudeicao_dest_media, direcaovento_dest_media, pressaoatmestacao_dest_media, tempbulboseco_dest_media, tempbulboumido_dest_media, umidaderelativa_dest_media, velocidadeventonebulosidade_dest_media, altitudeicao_origem_zero, direcaovento_origem_zero, pressaoatmestacao_origem_zero, tempbulboseco_origem_zero, tempbulboumido_origem_zero, umidaderelativa_origem_zero, velocidadeventonebulosidade_origem_zero, altitudeicao_dest_zero, direcaovento_dest_zero, pressaoatmestacao_dest_zero, tempbulboseco_dest_zero, tempbulboumido_dest_zero, umidaderelativa_dest_zero, velocidadeventonebulosidade_dest_zero from voos_historico_tratada_5 WHERE situacao_voo = "REALIZADO"')

In [8]:
#dataset = pd.read_csv('Voo.csv', encoding='utf-8', sep=',')#as_pandas(cursor_impala)
#dataset.to_csv('Voo.csv')
dataset = as_pandas(cursor_impala)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset.shape


In [ ]:
dim

In [9]:
# PANDAS: Verifica os tipos de dados de cada atributo
tipos = dataset.dtypes
#print(tipos)

In [10]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset.describe()
#print(resultado)

In [11]:
#TRATAMENTOS DE DADO
dataset_filtrado = dataset#.loc[dataset['situacao_voo'] == 'REALIZADO']

In [12]:
#DUMMIES
dummies_aero_org = pd.get_dummies(dataset_filtrado['cod_aero_org'], prefix='dummy_origem')
dummies_aero_dest = pd.get_dummies(dataset_filtrado['cod_aero_dest'], prefix='dummy_destino')
dummies_dia_semana_partida = pd.get_dummies(dataset_filtrado['dia_da_semana_partida'],prefix= 'dummy_dia_semana')


colunas_dummie_aero_org = list(dummies_aero_org)
colunas_dummie_aero_dest = list(dummies_aero_dest)
colunas_dummie_dia_semana_partida = list(dummies_dia_semana_partida)

dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_org], axis=1)
dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_dest], axis=1)
dataset_filtrado = pd.concat([dataset_filtrado, dummies_dia_semana_partida], axis=1)

In [13]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset_filtrado.shape
print(dim)

(2139536, 278)


In [14]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset_filtrado.describe()
print(resultado)

                  Unnamed: 0     flg_atraso_partida     flg_atraso_chegada  \
count  2.139536000000000e+06  2.139536000000000e+06  2.139536000000000e+06   
mean   1.069767500000000e+06  9.758798169322694e-02  1.170174280778636e-01   
std    6.176309871079980e+05  2.967568174160118e-01  3.214411266418867e-01   
min    0.000000000000000e+00  0.000000000000000e+00  0.000000000000000e+00   
25%    5.348837500000000e+05  0.000000000000000e+00  0.000000000000000e+00   
50%    1.069767500000000e+06  0.000000000000000e+00  0.000000000000000e+00   
75%    1.604651250000000e+06  0.000000000000000e+00  0.000000000000000e+00   
max    2.139535000000000e+06  1.000000000000000e+00  1.000000000000000e+00   

       feriado_federal_partida  feriado_federal_chegada  feriado_estadual_partida  \
count    2.139536000000000e+06    2.139536000000000e+06     2.139536000000000e+06   
mean     2.128779324115135e-02    2.128779324115135e-02     3.267998294957411e-03   
std      1.443420688434058e-01    1.443420

In [ ]:
# PANDAS: Distribuição das classses
contagem = dataset_filtrado.groupby('flg_atraso_partida').size()
print(contagem)

In [ ]:
# PANDAS: Correlação
cor = dataset_filtrado.corr(method='pearson')
print(cor)

In [ ]:
dataset_filtrado.shape

In [17]:

cols = ['feriado_federal_chegada', 'feriado_estadual_chegada', 'feriado_municipal_chegada', 'altitudeicao_origem_media', 'direcaovento_origem_media', 'pressaoatmestacao_origem_media', 'tempbulboseco_origem_media', 'tempbulboumido_origem_media'] + colunas_dummie_aero_dest + colunas_dummie_aero_org +colunas_dummie_dia_semana_partida

y_total = dataset_filtrado['flg_atraso_partida']
x_total = dataset_filtrado[cols]

train_x, test_x, train_y, test_y = train_test_split(
     dataset_filtrado[cols] , dataset_filtrado['flg_atraso_partida'], test_size=0.33, random_state=42)


In [ ]:
#REGRESSÃO LOGÍSTICA

clf_lr = linear_model.LogisticRegression(penalty='l2')
clf_lr.fit(train_x, train_y)

In [ ]:
predictions = clf_lr.predict(test_x)

In [ ]:
print(classification_report(test_y,predictions))

In [ ]:
print(accuracy_score(test_y,predictions))

In [18]:
#RANDOMIZED GRID SEARCH - RANDOM FOREST
n_estimators = [1, 10 , 25] #[1, 5, 10, 15, 20, 25]

max_features = ['auto', 'sqrt']

max_depth = [10,  50,  100, 500 ]


min_samples_split = [2, 5, 10, 20]

min_samples_leaf = [1,  3,  8,  21, 34, 55]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [19]:
#RANDOMGRIDSEARCH RANDOM FOREST#RANDOMG 
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(x_total, y_total)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV]  bootstrap=True, min_samples_l

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.8min


[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.3min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.3min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.3min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=500 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.3min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=500 
[CV]  bootstrap=False, min_s

[CV]  bootstrap=False, min_samples_leaf=55, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=100, total=  17.1s
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total= 1.9min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50, total= 1.0min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total= 1.9min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1, min_samples_split=20, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_

[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total= 1.5min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=500, total=  20.5s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=500, total=  19.1s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=500, total=  20.4s
[CV] bootstrap=False, min_samples_le

...

[CV]  bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=10, max_features=sqrt, max_depth=50, total= 1.2min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=10, max_features=sqrt, max_depth=50, total= 1.2min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=10, max_features=sqrt, max_depth=50, total= 1.2min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=10, max_features=sqrt, max_depth=50, total= 1.2min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=False, min_s

[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=100, total= 3.9min
[CV] bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10 
[CV]  bootstrap=True, min_samples_leaf=21, n_estimators=25, min_samples_split=5, max_features=sqrt, max_depth=10, total= 1.2min
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10, total=  43.1s
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=21, n_estimators=25, min_samples_split=5, max_features=sqrt, max_depth=10, total= 1.3min
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=

[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=  16.4s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=  16.6s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=  18.2s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=20, max_features=sqrt, max_depth=100 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=  17.0s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=20, max_features=sqrt, max_depth=100 
[CV]  bootstrap=False, min_samples

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 106.3min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 3, 8, 21, 34, 55], 'n_estimators': [1, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 50, 100, 500]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [39]:
print(rf_random.best_params_)

{'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 25, 'max_features': 'auto', 'min_samples_split': 10, 'max_depth': 10}


In [44]:
#RANDOM FOREST

#rfc = RandomForestClassifier(max_depth=None, random_state=42)

rfc = RandomForestClassifier(max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

rfc.fit(train_x, train_y)

#rfc = rf_random.best_estimator_.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
predictions = rfc.predict(test_x)

In [42]:
print(confusion_matrix(test_y,predictions))

[[637223      0]
 [ 68824      0]]


In [43]:
#best parameter
print(classification_report(test_y,predictions))

             precision    recall  f1-score   support

          0       0.90      1.00      0.95    637223
          1       0.00      0.00      0.00     68824

avg / total       0.81      0.90      0.86    706047



In [47]:
print(confusion_matrix(test_y,predictions))
print(classification_report(test_y,predictions))

[[613688  23535]
 [ 57389  11435]]
             precision    recall  f1-score   support

          0       0.91      0.96      0.94    637223
          1       0.33      0.17      0.22     68824

avg / total       0.86      0.89      0.87    706047



In [30]:
print(accuracy_score(test_y,predictions))

0.8850501453869218


In [25]:
print (sorted(zip(map(lambda x: round(x, 4), rfc.feature_importances_), cols), 
             reverse=True))

[(0.2501, 'pressaoatmestacao_origem_media'), (0.2339, 'tempbulboseco_origem_media'), (0.1776, 'tempbulboumido_origem_media'), (0.0731, 'direcaovento_origem_media'), (0.0112, 'altitudeicao_origem_media'), (0.0099, u'dummy_destino_SBBR'), (0.009, u'dummy_destino_SBCF'), (0.0088, u'dummy_destino_SBCT'), (0.0082, u'dummy_destino_SBGR'), (0.008, u'dummy_destino_SBKP'), (0.0074, u'dummy_destino_SBSV'), (0.0073, u'dummy_destino_SBPA'), (0.0072, u'dummy_destino_SBRF'), (0.0064, 'dummy_dia_semana_0'), (0.0063, 'dummy_dia_semana_1'), (0.0061, 'dummy_dia_semana_2'), (0.0061, u'dummy_destino_SBSP'), (0.0057, 'dummy_dia_semana_3'), (0.0054, u'dummy_destino_SBFZ'), (0.0052, 'dummy_dia_semana_5'), (0.0051, 'dummy_dia_semana_4'), (0.0051, u'dummy_destino_SBGO'), (0.005, u'dummy_destino_SBCY'), (0.0047, u'dummy_destino_SBFL'), (0.0046, u'dummy_destino_SBVT'), (0.0041, u'dummy_destino_SBBE'), (0.0038, 'feriado_federal_chegada'), (0.0038, u'dummy_destino_SBRJ'), (0.0038, u'dummy_destino_SBMO'), (0.0038, 

In [35]:
for feature in zip(cols, rfc.feature_importances_):
    print(feature)

('feriado_federal_chegada', 0.0038777998742642915)
('feriado_estadual_chegada', 0.00024336139826305572)
('feriado_municipal_chegada', 0.0016044590018078771)
('altitudeicao_origem_media', 0.010561166200801893)
('direcaovento_origem_media', 0.0824669364074239)
('pressaoatmestacao_origem_media', 0.2467281808915605)
('tempbulboseco_origem_media', 0.23144315929352208)
('tempbulboumido_origem_media', 0.17092511906043492)
(u'dummy_destino_SBAA', 2.766881005370411e-08)
(u'dummy_destino_SBAQ', 4.241018744944829e-08)
(u'dummy_destino_SBAR', 0.002827738308928248)
(u'dummy_destino_SBAT', 7.44830384896674e-05)
(u'dummy_destino_SBAU', 0.0006156131000527555)
(u'dummy_destino_SBAX', 0.00010561509706274375)
(u'dummy_destino_SBBE', 0.0039608040619212945)
(u'dummy_destino_SBBH', 0.0007408279775146645)
(u'dummy_destino_SBBR', 0.010512933571036073)
(u'dummy_destino_SBBV', 0.0004650594282762085)
(u'dummy_destino_SBBW', 0.00010802068375998477)
(u'dummy_destino_SBCA', 0.0014798331983049895)
(u'dummy_destino_S

In [48]:
sfm = SelectFromModel(rfc, threshold=0.15)
sfm.fit(train_x, train_y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        norm_order=1, prefit=False, threshold=0.15)

In [50]:
for feature_list_index in sfm.get_support(indices=True):
    print(cols[feature_list_index])

pressaoatmestacao_origem_media
tempbulboseco_origem_media
tempbulboumido_origem_media


In [51]:

cols = ['pressaoatmestacao_origem_media','tempbulboseco_origem_media','tempbulboumido_origem_media']

y_total = dataset_filtrado['flg_atraso_partida']
x_total = dataset_filtrado[cols]

train_x, test_x, train_y, test_y = train_test_split(
     dataset_filtrado[cols] , dataset_filtrado['flg_atraso_partida'], test_size=0.33, random_state=42)


In [52]:
#RANDOMGRIDSEARCH RANDOM FOREST#RANDOMG 
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(train_x, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV] bootstrap=False, min_samples_leaf=8, n_estimators=10, min_samples_split=20, max_features=auto, max_depth=50 
[CV]  bootstrap=True, min_samples_l

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.9min


[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.1min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.0min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=5, max_features=auto, max_depth=10, total= 1.0min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=500 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50, total= 2.2min
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=500 
[CV]  bootstrap=True, min_samp

[CV]  bootstrap=False, min_samples_leaf=55, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=100, total=   4.1s
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  41.2s
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  41.7s
[CV] bootstrap=True, min_samples_leaf=34, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  41.7s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1, min_samples_split=20, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_

[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1, min_samples_split=2, max_features=auto, max_depth=500, total=   5.3s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  51.2s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  51.4s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=50 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50, total=  50.5s
[CV]  bootstrap=True, min_samples_le

...

[CV]  bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=10, max_features=sqrt, max_depth=50, total=  51.4s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=True, min_samples_leaf=3, n_estimators=25, min_samples_split=2, max_features=sqrt, max_depth=10, total= 1.4min
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100, total=   3.8s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100, total=   4.0s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=10, max_features=auto, max_depth=100 
[CV]  bootstrap=False, mi

[CV]  bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10, total=  26.1s
[CV] bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10, total=  26.6s
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10, total=  26.1s
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=3, n_estimators=10, min_samples_split=2, max_features=sqrt, max_depth=10, total=  26.7s
[CV] bootstrap=True, min_samples_leaf=8, n_estimators=10, min_samples_split=5, max_features=sqrt, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf

[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=   4.0s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=   3.8s
[CV] bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=   4.2s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=20, max_features=sqrt, max_depth=100 
[CV]  bootstrap=False, min_samples_leaf=34, n_estimators=1, min_samples_split=5, max_features=auto, max_depth=50, total=   4.0s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=25, min_samples_split=20, max_features=sqrt, max_depth=100 
[CV]  bootstrap=False, min_samples

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 63.1min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 3, 8, 21, 34, 55], 'n_estimators': [1, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 50, 100, 500]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [56]:


#RANDOM FOREST

#rfc = RandomForestClassifier(max_depth=None, random_state=42)

rfc = rf_random.best_estimator_.fit(train_x, train_y)



In [57]:
predictions = rfc.predict(test_x)
print(confusion_matrix(test_y,predictions))
print(classification_report(test_y,predictions))

[[636008   1215]
 [ 67326   1498]]
             precision    recall  f1-score   support

          0       0.90      1.00      0.95    637223
          1       0.55      0.02      0.04     68824

avg / total       0.87      0.90      0.86    706047



In [58]:
print(accuracy_score(test_y,predictions))

0.9029228932351528
